In [1]:
using IRQIV
import CSV
using DataFrames
import PyPlot as plt
import Statistics#, NaNStatistics
import Geodesy
import Pkg
import Dates
import LaTeXStrings
plt.pygui(true) # sets plots to popup instead of inline

# Global camera parameters
fl = 1214.2857142857144 # In pixels - from Seth
include(raw"/Users/evanheberlein/Library/CloudStorage/Box-Box/Cornell/IRQIV.jl/examples/CameraCalibrationHelpers.jl")
# include(raw"/Users/evanheberlein/Library/CloudStorage/Box-Box/Cornell/IRQIV.jl/src/SpatialInterpolation.jl")
Camera = IRQIV.FLIR_SC8303_17mm
ι = IRQIV.IntrinsicCalibrationParameters(Camera)

# Use Pkg.develop(path="/Users/evanheberlein/Library/CloudStorage/Box-Box/Cornell/IRQIV.jl") to update IRQIV.jl (after pulling from repo)

Threads.nthreads() # Can run in 8 threads

8

In [2]:
# import Pkg
# Pkg.instantiate()
# Pkg.develop(path="/Users/evanheberlein/Library/CloudStorage/Box-Box/Cornell/IRQIV.jl")

In [3]:
# Import GCP data
DH_survey = CSV.read("/Users/evanheberlein/Library/CloudStorage/Box-Box/Cornell/CACO/2023/CornellDuckHarbor2023.csv", DataFrames.DataFrame)
GCPs_8_1_SC = DH_survey[16:64,:] # DataFrames makes first row headers


camlla_SC = GCPs_8_1_SC[37,7:9]
camlla_SC[3] += 2 # Guess elevation of 2m over camera spot
camutm_SC = Geodesy.UTMZ(
    Geodesy.LLA(camlla_SC[2], camlla_SC[1], camlla_SC[3]), Geodesy.wgs84
)

UTMZ(410794.4286334191, 4.644372350586758e6, -22.717, zone=19 (north))

In [4]:
SC_bots_uv_8_1 = [854 564 # SC1 bot
                751 108 # SC2 bot 
                426 64 # SC3 bot
                342 57 # SC4 bot
                287 51 # SC6 bot
                259 141 # SC8 bot
                242 132 # SC9 bot
                228 131 # SC10 bot
                230 226 # SC11 bot
                236 381 # SC12 bot
                344 639 # SC13 bot
                255 593 # NC1 bot
                266 656 # NC7 bot 
]

SC_tops_uv_8_1 = [622 602 # SC1 top
                550 114 # SC2 top
                318 63 # SC3 top
                273 61 # SC4 top
                236 53 # SC6 top
                230 63 # SC7 top
                228 141 # SC8 top
                218 133 # SC9 top
                205 131 # SC10 top
                217 227 # SC11 top
                218 381 # SC12 top
                #229 512 # Stump
                301 651 # SC13 top
                239 594 # NC1 top
                253 659 # NC7 top
]

SC_uv_8_1 = [SC_bots_uv_8_1
    SC_tops_uv_8_1]

27×2 Matrix{Int64}:
 854  564
 751  108
 426   64
 342   57
 287   51
 259  141
 242  132
 228  131
 230  226
 236  381
   ⋮  
 230   63
 228  141
 218  133
 205  131
 217  227
 218  381
 301  651
 239  594
 253  659

In [5]:
SC_bots_xyz_8_1 = GCPs_8_1_SC[25:36,[1,7,8,9]] # Name, Lat, long, elev
deleteat!(SC_bots_xyz_8_1, 6)
SC_bots_xyz_8_1 = [SC_bots_xyz_8_1
                GCPs_8_1_SC[1:7,[1,7,8,9]]]
deleteat!(SC_bots_xyz_8_1, (13:17))

SC_tops_xyz_8_1 = GCPs_8_1_SC[38:49,[1,7,8,9]] # "
SC_tops_xyz_8_1 = [SC_tops_xyz_8_1
                GCPs_8_1_SC[13:19,[1,7,8,9]]]
deleteat!(SC_tops_xyz_8_1, (14:18))

SC_xyz_8_1_lla = Matrix([SC_bots_xyz_8_1[:,2:4]
                SC_tops_xyz_8_1[:,2:4]])

SC_xyz_8_1_lla = Geodesy.LLA.(SC_xyz_8_1_lla[:,2], SC_xyz_8_1_lla[:,1], SC_xyz_8_1_lla[:,3])

# ChatGPT - can't use UTMZ.(Geodesy.LLA) for some reason, get into matrix outside of Geodesy.UTMZ structure
SC_xyz_8_1_utm = [Geodesy.UTMZ(lla, Geodesy.wgs84) for lla in SC_xyz_8_1_lla] 
SC_xyz_8_1_utm = hcat([utm.x for utm in SC_xyz_8_1_utm],
    [utm.y for utm in SC_xyz_8_1_utm],
    [utm.z for utm in SC_xyz_8_1_utm])


27×3 Matrix{Float64}:
      4.10794e5  4.64438e6  -25.688
      4.10797e5  4.64438e6  -25.74
      4.108e5    4.64438e6  -25.844
      4.10802e5  4.64439e6  -25.941
      4.10805e5  4.64439e6  -25.966
      4.10807e5  4.6444e6   -26.077
      4.1081e5   4.64441e6  -26.035
      4.10812e5  4.64441e6  -26.023
      4.10811e5  4.64442e6  -26.015
      4.10807e5  4.64443e6  -25.938
      ⋮                     
      4.10807e5  4.6444e6   -25.06
      4.10808e5  4.6444e6   -25.025
      4.1081e5   4.64441e6  -24.993
      4.10812e5  4.64441e6  -24.986
 410811.0        4.64442e6  -24.967
      4.10807e5  4.64443e6  -24.892
      4.10795e5  4.6444e6   -24.973
      4.10798e5  4.64445e6  -25.038
      4.10794e5  4.64444e6  -25.101

In [6]:
SC_bots_xyz_8_1 = GCPs_8_1_SC[25:36,[1,7,8,9]] # Name, Lat, long, elev


Row,Name,Longitude,Latitude,Ellipsoidal height
,String31,Float64,Float64,Float64
1,SC_stakes_bot_01,-70.0762,41.9463,-25.688
2,SC_stakes_bot_02,-70.0762,41.9463,-25.74
3,SC_stakes_bot_03,-70.0762,41.9464,-25.844
4,SC_stakes_bot_04,-70.0761,41.9464,-25.941
5,SC_stakes_bot_06,-70.0761,41.9465,-25.966
6,SC_stakes_bot_07,-70.0761,41.9465,-26.086
7,SC_stakes_bot_08,-70.0761,41.9466,-26.077
8,SC_stakes_bot_09,-70.076,41.9466,-26.035
9,SC_stakes_bot_10,-70.076,41.9467,-26.023


In [7]:
# Camera location calibration setup - south channel

# All converge for 8/1
azimuth_SC_g = deg2rad(5) # Guess from google maps
tilt_SC_g = deg2rad(-110) # From looking straight up?
roll_SC_g = deg2rad(90)

χinitial_SC = ExtrinsicCalibrationParameters(azimuth_SC_g, tilt_SC_g, roll_SC_g, fl, camutm_SC.x, camutm_SC.y, camutm_SC.z)

TrueGCPuv_SC_8_1 = hcat(RemoveRadialDistortion(SC_uv_8_1[:,1], SC_uv_8_1[:,2], Camera)...)

plt.close("all")
GCP_path_SC_8_1 = ("/Users/evanheberlein/Library/CloudStorage/Box-Box/Cornell/CACO_backup/2023/DH_IR/Rec-HRE_2023_-003701-__TIME__.ats")
GCP_image_SC_8_1 = LoadATSImageSequence(GCP_path_SC_8_1, 1, 2023)
GraphicallyExploreInitialCalibrationGuess(TrueGCPuv_SC_8_1, SC_xyz_8_1_utm, χinitial_SC, ι, GCP_image_SC_8_1[1])
plt.show()




┌ Warning: Too few BHPHeaders to calculate fps!
└ @ IRQIV.FLIRATSFiles /Users/evanheberlein/Library/CloudStorage/Box-Box/Cornell/IRQIV.jl/src/FLIRATSFiles.jl:415
┌ Info: (410794.4845426629, 410794.0567097684)
└ @ Main /Users/evanheberlein/Library/CloudStorage/Box-Box/Cornell/IRQIV.jl/examples/CameraCalibrationHelpers.jl:68


┌ Info: (410796.5026560258, 410796.72707823775)
└ @ Main /Users/evanheberlein/Library/CloudStorage/Box-Box/Cornell/IRQIV.jl/examples/CameraCalibrationHelpers.jl:68
┌ Info: (410799.7935985873, 410801.4967871165)
└ @ Main /Users/evanheberlein/Library/CloudStorage/Box-Box/Cornell/IRQIV.jl/examples/CameraCalibrationHelpers.jl:68
┌ Info: (410802.3085480358, 410807.28713693673)
└ @ Main /Users/evanheberlein/Library/CloudStorage/Box-Box/Cornell/IRQIV.jl/examples/CameraCalibrationHelpers.jl:68
┌ Info: (410805.3732788639, 410819.7747427694)
└ @ Main /Users/evanheberlein/Library/CloudStorage/Box-Box/Cornell/IRQIV.jl/examples/CameraCalibrationHelpers.jl:68
┌ Info: (410807.4250922289, 410832.1277337212)
└ @ Main /Users/evanheberlein/Library/CloudStorage/Box-Box/Cornell/IRQIV.jl/examples/CameraCalibrationHelpers.jl:68
┌ Info: (410809.6205921579, 410873.4262477472)
└ @ Main /Users/evanheberlein/Library/CloudStorage/Box-Box/Cornell/IRQIV.jl/examples/CameraCalibrationHelpers.jl:68
┌ Info: (410812.3437

┌ Info: (850.6949468623131, 945.0580834711756)
└ @ Main /Users/evanheberlein/Library/CloudStorage/Box-Box/Cornell/IRQIV.jl/examples/CameraCalibrationHelpers.jl:104
┌ Info: (749.6575863776918, 915.2333048944645)
└ @ Main /Users/evanheberlein/Library/CloudStorage/Box-Box/Cornell/IRQIV.jl/examples/CameraCalibrationHelpers.jl:104
┌ Info: (429.1250136179358, 575.406219837551)
└ @ Main /Users/evanheberlein/Library/CloudStorage/Box-Box/Cornell/IRQIV.jl/examples/CameraCalibrationHelpers.jl:104
┌ Info: (345.0285257261116, 482.2750755223382)
└ @ Main /Users/evanheberlein/Library/CloudStorage/Box-Box/Cornell/IRQIV.jl/examples/CameraCalibrationHelpers.jl:104
┌ Info: (289.28130022350297, 416.4534758180706)
└ @ Main /Users/evanheberlein/Library/CloudStorage/Box-Box/Cornell/IRQIV.jl/examples/CameraCalibrationHelpers.jl:104
┌ Info: (262.418780637071, 367.64005129447054)
└ @ Main /Users/evanheberlein/Library/CloudStorage/Box-Box/Cornell/IRQIV.jl/examples/CameraCalibrationHelpers.jl:104
┌ Info: (244.949

In [8]:
# Camera location calibration - south channel
dlt_us = IRQIV.DLT(χinitial_SC, FLIR_SC8303_17mm)

χcalculated_SC_8_1 = ExtrinsicCalibration(
    TrueGCPuv_SC_8_1,
    SC_xyz_8_1_utm,
    χinitial_SC,
    ι,
    MaxIterations = 50,
    PrintoutStats=true)

@info "Initial guess, χinitial:"
@show dump(χinitial_SC)
# 
@info "Calibrated values, χcalculated:"
@show dump(χcalculated_SC_8_1)
# 
# Describe how much the calibration changed each parameter:
@info "Difference between calibrated value and initial guess:"
for (i,x) in enumerate(fieldnames(typeof(χcalculated_SC_8_1)))
    if i <= 3
        println("$x diff (degrees) -  $(rad2deg(getfield(χinitial_SC, x) - getfield(χcalculated_SC_8_1, x)))")
    elseif i == 4
        println("$x diff -  $(getfield(χinitial_SC, x) - getfield(χcalculated_SC_8_1, x))")
    else
        println("$x diff (meters) -  $(getfield(χinitial_SC, x) - getfield(χcalculated_SC_8_1, x))")
    end
end

plt.close("all")
GraphicallyExploreInitialCalibrationGuess(TrueGCPuv_SC_8_1, SC_xyz_8_1_utm, χcalculated_SC_8_1, ι, GCP_image_SC_8_1[1])
plt.show()

Finished extrinsic calibration:, 8 iterations, max(|ε|)=5e-11, rms(ε₁-ε₀)=7e-11


┌ Info: Initial guess, χinitial:
└ @ Main /Users/evanheberlein/Library/CloudStorage/Box-Box/Cornell/CACO/CACO_git/CACO_23_jl/DH_georef.ipynb:12


ExtrinsicCalibrationParameters
  φ: Float64 0.08726646259971647
  τ: Float64 -1.9198621771937625
  σ: Float64 1.5707963267948966
  f: Float64 1214.2857142857144
  x_c: Float64 410794.4286334191
  y_c: Float64 4.644372350586758e6
  z_c: Float64 -22.717
dump(χinitial_SC) = nothing
ExtrinsicCalibrationParameters
  φ: Float64 0.15869663905804743
  τ: Float64 -1.9723979042818223
  σ: Float64 1.7056864953526294
  f: Float64 1155.546818910951
  x_c: Float64 410794.6490761117
  y_c: Float64 4.644372713219419e6
  z_c: Float64 -23.205442559739787
dump(χcalculated_SC_8_1) = nothing
φ diff (degrees) -  -4.092647640937094
τ diff (degrees) -  3.010075435796938
σ diff (degrees) -  -7.72863735616637
f diff -  58.73889537476339
x_c diff (meters) -  -0.22044269257457927
y_c diff (meters) -  -0.362632661126554
z_c diff (meters) -  0.4884425597397879


┌ Info: Calibrated values, χcalculated:
└ @ Main /Users/evanheberlein/Library/CloudStorage/Box-Box/Cornell/CACO/CACO_git/CACO_23_jl/DH_georef.ipynb:15
┌ Info: Difference between calibrated value and initial guess:
└ @ Main /Users/evanheberlein/Library/CloudStorage/Box-Box/Cornell/CACO/CACO_git/CACO_23_jl/DH_georef.ipynb:19


┌ Info: (410794.4845426629, 410794.5197265509)
└ @ Main /Users/evanheberlein/Library/CloudStorage/Box-Box/Cornell/IRQIV.jl/examples/CameraCalibrationHelpers.jl:68
┌ Info: (410796.5026560258, 410796.5553979582)
└ @ Main /Users/evanheberlein/Library/CloudStorage/Box-Box/Cornell/IRQIV.jl/examples/CameraCalibrationHelpers.jl:68
┌ Info: (410799.7935985873, 410799.7594660288)
└ @ Main /Users/evanheberlein/Library/CloudStorage/Box-Box/Cornell/IRQIV.jl/examples/CameraCalibrationHelpers.jl:68
┌ Info: (410802.3085480358, 410802.34001171717)
└ @ Main /Users/evanheberlein/Library/CloudStorage/Box-Box/Cornell/IRQIV.jl/examples/CameraCalibrationHelpers.jl:68
┌ Info: (410805.3732788639, 410805.41605494975)
└ @ Main /Users/evanheberlein/Library/CloudStorage/Box-Box/Cornell/IRQIV.jl/examples/CameraCalibrationHelpers.jl:68
┌ Info: (410807.4250922289, 410807.34820047667)
└ @ Main /Users/evanheberlein/Library/CloudStorage/Box-Box/Cornell/IRQIV.jl/examples/CameraCalibrationHelpers.jl:68
┌ Info: (410809.620

In [9]:
SC4_bot_z = SC_bots_xyz_8_1[4,4]
SC6_bot_z = SC_bots_xyz_8_1[5,4]
SC6_bot_z = SC_bots_xyz_8_1[6,4]
SC4_bot_z - χcalculated_SC_8_1.z_c
# SC6_bot_z - χcalculated_SC_8_1.z_c
# SC_bots_xyz_8_1

-2.7355574402602123

In [10]:
# File numbers & water levels:
# 3714 - SC8 bot (people in frame at beginning)
# 3717 - SC6 bot
# 3723 - SC4 bot
# 3733 (last file) b/w SC3 bot & SC4 bot

In [11]:
# Time series of rectified image
SC4_bot_z = SC_bots_xyz_8_1[4,4] # WSE
interpolated_grid_spacing=.1
RectParams = IRQIV.ImageRectification.calculate_image_rectification(Camera, χcalculated_SC_8_1, 
SC4_bot_z; 
interpolated_grid_spacing,
u_limits = (290, 900))

# Select IR image file:
FLIR_filename = raw"/Volumes/ETH_4TB/DH_IR/Rec-HRE_2023_-003733-__TIME__.ats"
# Load IR images (from Sutter Slough):
nfiles = 300
IRimages = LoadATSImageSequence(FLIR_filename, nfiles, 2023)
far_field_exclude = 375#350 # How many rows to remove from top of rectified image

# Rectify a single image
plt.close("all")
fig = plt.figure(figsize=(9, 9))
for i in 1:length(IRimages)
    rectified_image = RectParams(IRimages[i])[far_field_exclude:end-50,:]
    #
    ax = plt.gca()

    ax.imshow(rectified_image;extent=RectParams.extent)
    plt.pause(.1)
    ax.cla() # Clear figure
    plt.xlabel("UTM z19N easting (m)")
    plt.ylabel("UTM z19N northing (m)")
    # ax.axhline(STIV_row+far_field_exclude, color="red", linestyle="--", linewidth=2) # Changes figure dimensions, imshow disappears

    # function annotate_pixel_coordinates(event) # Trying to get pixel coordinates in imshow, not working
    #     x, y = round(event.x), round(event.y)
    #     ax.annotate("($x, $y)", xy=(event.x, event.y), xytext=(x + 1, y + 1), color="white", fontsize=8)
    #     fig.canvas.draw_idle()  # Force redraw to update the annotation
    # end
    # connect(plt.gcf(), "button_press_event", annotate_pixel_coordinates)


    # Update rectified image to last image in data set, plot this as well:
    # rectified_image = RectParams(rectified_image, IRimages[end])
    # ax[2].imshow(rectified_image;extent=RectParams.extent)
    ax.set_title("Datetime: $(ats_ts(IRimages)[i])")
    ###############################################################################
    # plt.show()
end

  0.004913 seconds (41.82 k allocations: 4.148 MiB)


In [16]:
STIV_row = 20

# STIV
plt.close("all")
fig = plt.figure(figsize=(7,5))
ax = plt.gca()
# Select IR image file:
FLIR_filename = raw"/Volumes/ETH_4TB/DH_IR/Rec-HRE_2023_-003723-__TIME__.ats"
# Load IR images (from Sutter Slough):
nfiles = 300
IRimages = LoadATSImageSequence(FLIR_filename, nfiles, 2023)
rectified_image = RectParams(IRimages[1])[far_field_exclude:end,:]
STIV_fig = zeros(size(rectified_image)[2], length(IRimages)).*NaN


nticks = 6
# tick_int = STIV_yticks/(nticks-1)
# tick_labels = 
###############################################################################

for i in 1:length(IRimages)
    rectified_image = RectParams(IRimages[i])[far_field_exclude:end,:]
    STIV_fig[:,i] = rectified_image[STIV_row,:]

    ###############################################################################
    # plt.show()
end

function remove_nans(row)
    return filter(!isnan, row)
end
STIV_real = hcat(map(remove_nans, eachcol(STIV_fig))...)
nanmask_col = .!isnan.(STIV_fig[:,1])

# Create yticks - spatial
STIV_y_coords = RectParams.UTMxi'[STIV_row+far_field_exclude,:][nanmask_col] # Get UTM coordinates in x for STIV y-axis label
total_ticks = size(STIV_real)[1]
total_distance_m = STIV_y_coords[end] - STIV_y_coords[1]
tick_increment = total_distance_m/total_ticks * 20
yticks_m = 0:tick_increment:(total_distance_m - 1e-10)  # Adjusted to avoid floating-point precision issues
yticks_m = round.(yticks_m, digits=1)
yticks_ind = round.(Int, yticks_m ./ (total_distance_m / (total_ticks - 1)))

# Create xticks - temporal
total_axis_length = size(STIV_real)[2]
dt = 1/IRimages.Metadata.Estimatedfps
num_labels = 10
xticks_s = range(0, stop=(total_axis_length - 1) * dt, length=num_labels)
xticks_s = round.(xticks_s, digits= 2)
xticks_ind = round.(Int, xticks_s / dt) .+ 1

# STIV segments:
wave_segs = [[(0,46), (80,129)], [(51,0), (169,129)], [(203.6,66.9), (258,129)]]
# bulk_segs = [[(51.4,57.1), (116.2,88)], [(85.5,30.5), (136.1,51.7)], [(177,41), (299,86)]]
bulk_segs = [[(42.3,46.4), (184.2,104.3)],[(114.8,42.7),(299,124.7)],[(177,39), (299,84)]]

plt.close("all")
fig = plt.figure(figsize=(9,5))
ax = plt.gca()
ax.imshow(STIV_real)
i=1
for segment in wave_segs
    x_vals, y_vals = zip(segment...)
    if i == 1
        plt.plot(x_vals, y_vals, color="red", linewidth=2, linestyle="--", label="Wave celerity")
    else
        plt.plot(x_vals, y_vals, color="red", linewidth=2, linestyle="--")
    end
    i +=1
end
i=1
for segment in bulk_segs
    x_vals, y_vals = zip(segment...)
    if i == 1
        plt.plot(x_vals, y_vals, color="white", linewidth=2, linestyle="-.", label="Bulk velocity")
    else
        plt.plot(x_vals, y_vals, color="white", linewidth=2, linestyle="-.")
    end
    i +=1
end
plt.yticks(yticks_ind, string.(yticks_m, " m"))
plt.xticks(xticks_ind, string.(xticks_s, " s"))
plt.xlabel("Time (s)")
plt.ylabel("Distance (m)")
plt.legend()
plt.show()

# Calculate slope of defined segments
function calculate_slope(point1, point2)
    x1, y1 = point1
    x2, y2 = point2
    
    if x2 == x1
        return Inf  # Vertical line, undefined slope
    else
        return (y2 - y1) / (x2 - x1)
    end
end
dx = total_distance_m/total_ticks
dt = 0.05

wave_vels = zeros(length(wave_segs))
bulk_vels = zeros(length(bulk_segs))

for (i, segment) in enumerate(wave_segs)
    wave_vels[i] = calculate_slope(segment[1], segment[2])*(dx/dt)
end

for (i, segment) in enumerate(bulk_segs)
    bulk_vels[i] = calculate_slope(segment[1], segment[2])*(dx/dt)
end

wave_cs = wave_vels.-bulk_vels
wave_hs = (wave_cs.^2)/9.81
wave_cs_mean = mean(wave_vels)-mean(bulk_vels)
wave_hs_mean = wave_cs_mean^2/9.81

0.18751578518894377

In [31]:
dump(χcalculated_SC_8_1)
rad2deg(χcalculated_SC_8_1.τ)

ExtrinsicCalibrationParameters
  φ: Float64 0.15869663905804743
  τ: Float64 -1.9723979042818223
  σ: Float64 1.7056864953526294
  f: Float64 1155.546818910951
  x_c: Float64 410794.6490761117
  y_c: Float64 4.644372713219419e6
  z_c: Float64 -23.205442559739787


-113.01007543579694